In [74]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

In [75]:
daily_sales_numbers = [ 21,22,-108,31,-1,32,34,31]

In [76]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [77]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [78]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [79]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset:
    print(sales.numpy())

21
22
31
32
34
31


In [80]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset:
    print(sales.numpy())

1512
1584
2232
2304
2448
2232


In [81]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset:
    print(sales.numpy())

1512
1584
2232
2232
2304
2448


In [82]:
for sales in tf_dataset.batch(4):
    print(sales.numpy())

[2232 2304 1584 2448]
[1512 2232]


In [83]:
images_ds = tf.data.Dataset.list_files('PlantVillage/*/*',shuffle=False)
for file in images_ds.take(5):
    print(file.numpy())

b'PlantVillage\\Potato___Early_blight\\001187a0-57ab-4329-baff-e7246a9edeb0___RS_Early.B 8178.JPG'
b'PlantVillage\\Potato___Early_blight\\002a55fb-7a3d-4a3a-aca8-ce2d5ebc6925___RS_Early.B 8170.JPG'
b'PlantVillage\\Potato___Early_blight\\009c8c31-f22d-4ffd-8f16-189c6f06c577___RS_Early.B 7885.JPG'
b'PlantVillage\\Potato___Early_blight\\00d8f10f-5038-4e0f-bb58-0b885ddc0cc5___RS_Early.B 8722.JPG'
b'PlantVillage\\Potato___Early_blight\\0182e991-97f0-4805-a1f7-6e1b4306d518___RS_Early.B 7015.JPG'


In [84]:
images_ds = images_ds.shuffle(2000)
for file in images_ds.take(5):
    print(file.numpy())

b'PlantVillage\\Potato___Late_blight\\33551627-5824-42b9-9547-4df44590fee1___RS_LB 4848.JPG'
b'PlantVillage\\Potato___Late_blight\\d1ae329b-19bc-40c3-afb1-404ecf3c98ee___RS_LB 2613.JPG'
b'PlantVillage\\Potato___Late_blight\\b2492a9e-c038-48a7-8524-8f0aca7dc870___RS_LB 4291.JPG'
b'PlantVillage\\Potato___Late_blight\\29dafc9e-eb0f-4f3b-99ab-a617b8163e50___RS_LB 3052.JPG'
b'PlantVillage\\Potato___Early_blight\\c3bf3f15-b1ae-4e36-97a4-f341879db6ae___RS_Early.B 8263.JPG'


In [85]:
image_count = len(images_ds)
image_count

2152

In [86]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [87]:
len(train_ds)


1721

In [88]:
len(test_ds)

431

In [89]:
s = 'PlantVillage\\Potato___Early_blight\\eb4fbc2b-c1a4-473f-a4e1-71ba04a7133c___RS_Early.B 7716.JPG'
s.split("\\")[-2]

'Potato___Early_blight'

In [90]:
def get_label(file_path):
    import os
    return tf.strings.split(file_path, os.path.sep)[-2]

In [91]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    return img,label

In [93]:
train_ds = train_ds.map(process_image)
for img,label in train_ds.take(3):
    print(img)

tf.Tensor(
[[[181 163 161]
  [190 172 170]
  [186 168 166]
  ...
  [179 161 161]
  [186 168 168]
  [185 167 167]]

 [[178 160 158]
  [188 170 168]
  [180 162 160]
  ...
  [183 165 165]
  [187 169 169]
  [188 170 170]]

 [[163 145 143]
  [181 163 161]
  [182 164 162]
  ...
  [185 167 167]
  [184 166 166]
  [187 169 169]]

 ...

 [[140 112 109]
  [142 114 111]
  [159 131 128]
  ...
  [148 118 116]
  [130 100  98]
  [137 107 105]]

 [[150 122 119]
  [142 114 111]
  [155 127 124]
  ...
  [158 128 126]
  [131 101  99]
  [150 120 118]]

 [[158 130 127]
  [139 111 108]
  [148 120 117]
  ...
  [156 126 124]
  [129  99  97]
  [162 132 130]]], shape=(256, 256, 3), dtype=uint8)
tf.Tensor(
[[[108 109 127]
  [135 136 154]
  [120 121 139]
  ...
  [115 117 130]
  [113 115 128]
  [115 117 130]]

 [[137 138 156]
  [111 112 130]
  [118 119 137]
  ...
  [108 110 123]
  [112 114 127]
  [117 119 132]]

 [[123 124 142]
  [102 103 121]
  [119 120 138]
  ...
  [109 111 124]
  [114 116 129]
  [117 119 132]]

 

In [94]:
def scale(image,label):
    return image/255, label

In [96]:
train_ds = train_ds.map(scale)
for image,label in train_ds.take(5):
    print("*** IMAGE ", image.numpy()[0][0])
    print("*** LABEL ",label.numpy())

*** IMAGE  [0.5803922 0.5372549 0.5686275]
*** LABEL  b'Potato___Late_blight'
*** IMAGE  [0.6392157 0.6313726 0.6862745]
*** LABEL  b'Potato___Early_blight'
*** IMAGE  [0.7529412  0.74509805 0.79607844]
*** LABEL  b'Potato___Early_blight'
*** IMAGE  [0.53333336 0.5137255  0.5372549 ]
*** LABEL  b'Potato___Late_blight'
*** IMAGE  [0.76862746 0.75686276 0.78431374]
*** LABEL  b'Potato___Early_blight'
